In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

## Predictions for April 9th, 2024: 

In [2]:
initial_predictions_dataset = pd.read_csv("./test_data/Professor_CSV.csv")

In [3]:
initial_predictions_dataset.head(10)

,DATE,DAY,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,DEPARTURE TIME,ARRIVAL TIME,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late
0,4/19/24,FRIDAY,UA 1400,UA,UA,ORD,6:52 PM,9:47 PM,NaN,NaN,NaN,NaN
1,4/19/24,FRIDAY,AA 3402,AA,MQ,ORD,7:59 PM,10:52 PM,NaN,NaN,NaN,NaN
2,4/19/24,FRIDAY,B6 116,B6,B6,JFK,1:34 PM,2:51 PM,NaN,NaN,NaN,NaN
3,4/19/24,FRIDAY,DL 5182,DL,9E,JFK,2:55 PM,4:21 PM,NaN,NaN,NaN,NaN
4,4/19/24,FRIDAY,WN 5285,WN,WN,MCO,11:35 AM,2:20 PM,NaN,NaN,NaN,NaN
5,4/19/24,FRIDAY,B6 656,B6,B6,MCO,1:35 PM,4:25 PM,NaN,NaN,NaN,NaN
6,4/20/24,SATURDAY,UA 1400,UA,UA,ORD,6:52 PM,9:47 PM,NaN,NaN,NaN,NaN
7,4/20/24,SATURDAY,AA 3402,AA,MQ,ORD,7:59 PM,10:52 PM,NaN,NaN,NaN,NaN
8,4/20/24,SATURDAY,B6 116,B6,B6,JFK,1:25 PM,2:41 PM,NaN,NaN,NaN,NaN
9,4/20/24,SATURDAY,DL 5182,DL,9E,JFK,2:55 PM,4:21 PM,NaN,NaN,NaN,NaN


In [4]:
initial_predictions_dataset.columns

Index(['DATE', 'DAY', 'FLIGHT NUMBER', 'MKT_UNIQUE_CARRIER',
       'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEPARTURE TIME', 'ARRIVAL TIME',
       'ARRIVAL STATUS', 'ARRIVAL STATUS_Prev_flight_early',
       'ARRIVAL STATUS_Prev_flight_ontime', 'ARRIVAL STATUS_Prev_flight_late'],
      dtype='object')

In [5]:
initial_predictions_dataset.dtypes

DATE                                  object
DAY                                   object
FLIGHT NUMBER                         object
MKT_UNIQUE_CARRIER                    object
OP_UNIQUE_CARRIER                     object
ORIGIN                                object
DEPARTURE TIME                        object
ARRIVAL TIME                          object
ARRIVAL STATUS                       float64
ARRIVAL STATUS_Prev_flight_early     float64
ARRIVAL STATUS_Prev_flight_ontime    float64
ARRIVAL STATUS_Prev_flight_late      float64
dtype: object

In [6]:
def convert_to_24hr_format(date, time):
    # Adjust the format to match the 'month/day/year' structure
    return pd.to_datetime(date + ' ' + time, format='%m/%d/%y %I:%M %p')

initial_predictions_dataset['SCH_DEP_TIME'] = initial_predictions_dataset.apply(
    lambda row: convert_to_24hr_format(row['DATE'], row['DEPARTURE TIME']), axis=1)
initial_predictions_dataset['SCH_ARR_TIME'] = initial_predictions_dataset.apply(
    lambda row: convert_to_24hr_format(row['DATE'], row['ARRIVAL TIME']), axis=1)

initial_predictions_dataset.head(10)

,DATE,DAY,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,DEPARTURE TIME,ARRIVAL TIME,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME
0,4/19/24,FRIDAY,UA 1400,UA,UA,ORD,6:52 PM,9:47 PM,NaN,NaN,NaN,NaN,2024-04-19 18:52:00,2024-04-19 21:47:00
1,4/19/24,FRIDAY,AA 3402,AA,MQ,ORD,7:59 PM,10:52 PM,NaN,NaN,NaN,NaN,2024-04-19 19:59:00,2024-04-19 22:52:00
2,4/19/24,FRIDAY,B6 116,B6,B6,JFK,1:34 PM,2:51 PM,NaN,NaN,NaN,NaN,2024-04-19 13:34:00,2024-04-19 14:51:00
3,4/19/24,FRIDAY,DL 5182,DL,9E,JFK,2:55 PM,4:21 PM,NaN,NaN,NaN,NaN,2024-04-19 14:55:00,2024-04-19 16:21:00
4,4/19/24,FRIDAY,WN 5285,WN,WN,MCO,11:35 AM,2:20 PM,NaN,NaN,NaN,NaN,2024-04-19 11:35:00,2024-04-19 14:20:00
5,4/19/24,FRIDAY,B6 656,B6,B6,MCO,1:35 PM,4:25 PM,NaN,NaN,NaN,NaN,2024-04-19 13:35:00,2024-04-19 16:25:00
6,4/20/24,SATURDAY,UA 1400,UA,UA,ORD,6:52 PM,9:47 PM,NaN,NaN,NaN,NaN,2024-04-20 18:52:00,2024-04-20 21:47:00
7,4/20/24,SATURDAY,AA 3402,AA,MQ,ORD,7:59 PM,10:52 PM,NaN,NaN,NaN,NaN,2024-04-20 19:59:00,2024-04-20 22:52:00
8,4/20/24,SATURDAY,B6 116,B6,B6,JFK,1:25 PM,2:41 PM,NaN,NaN,NaN,NaN,2024-04-20 13:25:00,2024-04-20 14:41:00
9,4/20/24,SATURDAY,DL 5182,DL,9E,JFK,2:55 PM,4:21 PM,NaN,NaN,NaN,NaN,2024-04-20 14:55:00,2024-04-20 16:21:00


In [7]:
initial_predictions_dataset.drop(columns=['DATE', 'DEPARTURE TIME', 'ARRIVAL TIME'], inplace = True) 
initial_predictions_dataset.head()

,DAY,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME
0,FRIDAY,UA 1400,UA,UA,ORD,NaN,NaN,NaN,NaN,2024-04-19 18:52:00,2024-04-19 21:47:00
1,FRIDAY,AA 3402,AA,MQ,ORD,NaN,NaN,NaN,NaN,2024-04-19 19:59:00,2024-04-19 22:52:00
2,FRIDAY,B6 116,B6,B6,JFK,NaN,NaN,NaN,NaN,2024-04-19 13:34:00,2024-04-19 14:51:00
3,FRIDAY,DL 5182,DL,9E,JFK,NaN,NaN,NaN,NaN,2024-04-19 14:55:00,2024-04-19 16:21:00
4,FRIDAY,WN 5285,WN,WN,MCO,NaN,NaN,NaN,NaN,2024-04-19 11:35:00,2024-04-19 14:20:00


In [8]:
# Define a mapping from day names to numbers
day_to_number = {
    'MONDAY': 1,
    'TUESDAY': 2,
    'WEDNESDAY': 3,
    'THURSDAY': 4,
    'FRIDAY': 5,
    'SATURDAY': 6,
    'SUNDAY': 7
}

# Apply the mapping to the 'DAY' column to create 'DAY_OF_WEEK'
initial_predictions_dataset['DAY_OF_WEEK'] = initial_predictions_dataset['DAY'].map(day_to_number)
initial_predictions_dataset.drop(columns=['DAY'], inplace=True)
# Verify the changes
initial_predictions_dataset.head()


,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME,DAY_OF_WEEK
0,UA 1400,UA,UA,ORD,NaN,NaN,NaN,NaN,2024-04-19 18:52:00,2024-04-19 21:47:00,5
1,AA 3402,AA,MQ,ORD,NaN,NaN,NaN,NaN,2024-04-19 19:59:00,2024-04-19 22:52:00,5
2,B6 116,B6,B6,JFK,NaN,NaN,NaN,NaN,2024-04-19 13:34:00,2024-04-19 14:51:00,5
3,DL 5182,DL,9E,JFK,NaN,NaN,NaN,NaN,2024-04-19 14:55:00,2024-04-19 16:21:00,5
4,WN 5285,WN,WN,MCO,NaN,NaN,NaN,NaN,2024-04-19 11:35:00,2024-04-19 14:20:00,5


In [9]:

# Negative "diff_arrival_departure" indicates that flight arrive next day.
diff_arrival_departure = initial_predictions_dataset['SCH_ARR_TIME'] - initial_predictions_dataset['SCH_DEP_TIME']


initial_predictions_dataset = initial_predictions_dataset[diff_arrival_departure > pd.Timedelta(0)]

initial_predictions_dataset.shape
initial_predictions_dataset.dtypes

(23, 11)

FLIGHT NUMBER                                object
MKT_UNIQUE_CARRIER                           object
OP_UNIQUE_CARRIER                            object
ORIGIN                                       object
ARRIVAL STATUS                              float64
ARRIVAL STATUS_Prev_flight_early            float64
ARRIVAL STATUS_Prev_flight_ontime           float64
ARRIVAL STATUS_Prev_flight_late             float64
SCH_DEP_TIME                         datetime64[ns]
SCH_ARR_TIME                         datetime64[ns]
DAY_OF_WEEK                                   int64
dtype: object

In [10]:
initial_predictions_dataset['DEST'] = 'SYR'

In [11]:
weather_data = pd.read_csv('./test_data/Test_Weather_Data.csv', parse_dates=['datetime'])
weather_data.head()
weather_data.shape
weather_data.dtypes

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,Chicago,2024-04-18 00:00:00,55.70,55.70,41.60,58.97,0.00,0.00,NaN,0,0,25.30,12.40,301.00,"1,014.50",68.00,9.90,0,0.00,0,10,Partially cloudy,partly-cloudy-night,"KORD,KMDW,F1983,KPWK"
1,Chicago,2024-04-18 01:00:00,53.80,53.80,42.20,64.58,0.00,0.00,NaN,0,0,20.80,8.20,290.00,"1,015.10",32.40,9.90,0,0.00,0,10,Partially cloudy,partly-cloudy-night,"KORD,KMDW,F1983,KPWK"
2,Chicago,2024-04-18 02:00:00,51.30,51.30,42.20,70.94,0.00,0.00,NaN,0,0,17.20,3.70,267.00,"1,015.30",22.10,9.90,0,0.00,0,10,Partially cloudy,partly-cloudy-night,"KORD,KMDW,F1983,KPWK"
3,Chicago,2024-04-18 03:00:00,49.80,47.80,42.20,74.94,0.00,0.00,NaN,0,0,13.90,5.30,258.00,"1,015.20",28.20,9.90,0,0.00,0,10,Partially cloudy,partly-cloudy-night,"KORD,KMDW,F1983,KPWK"
4,Chicago,2024-04-18 04:00:00,49.30,46.60,42.20,76.33,0.00,0.00,NaN,0,0,10.30,6.40,282.00,"1,015.90",22.10,9.90,0,0.00,0,10,Partially cloudy,partly-cloudy-night,"KORD,KMDW,F1983,KPWK"


(1440, 24)

name                        object
datetime            datetime64[ns]
temp                       float64
feelslike                  float64
dew                        float64
humidity                   float64
precip                     float64
precipprob                 float64
preciptype                  object
snow                         int64
snowdepth                    int64
windgust                   float64
windspeed                  float64
winddir                    float64
sealevelpressure           float64
cloudcover                 float64
visibility                 float64
solarradiation               int64
solarenergy                float64
uvindex                      int64
severerisk                   int64
conditions                  object
icon                        object
stations                    object
dtype: object

In [12]:
print(set(weather_data['name']))

{'orlando', 'Chicago', 'syracuse', 'New York'}


In [13]:
# Replacing city names with airport names
dic = {'syracuse': 'SYR', 'new york': 'JFK', 'orlando': 'MCO', 'chicago': 'ORD'}

weather_data['name'] = weather_data['name'].apply(lambda row: dic[row.lower()])

weather_data.head()
print(set(weather_data['name']))

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,ORD,2024-04-18 00:00:00,55.70,55.70,41.60,58.97,0.00,0.00,NaN,0,0,25.30,12.40,301.00,"1,014.50",68.00,9.90,0,0.00,0,10,Partially cloudy,partly-cloudy-night,"KORD,KMDW,F1983,KPWK"
1,ORD,2024-04-18 01:00:00,53.80,53.80,42.20,64.58,0.00,0.00,NaN,0,0,20.80,8.20,290.00,"1,015.10",32.40,9.90,0,0.00,0,10,Partially cloudy,partly-cloudy-night,"KORD,KMDW,F1983,KPWK"
2,ORD,2024-04-18 02:00:00,51.30,51.30,42.20,70.94,0.00,0.00,NaN,0,0,17.20,3.70,267.00,"1,015.30",22.10,9.90,0,0.00,0,10,Partially cloudy,partly-cloudy-night,"KORD,KMDW,F1983,KPWK"
3,ORD,2024-04-18 03:00:00,49.80,47.80,42.20,74.94,0.00,0.00,NaN,0,0,13.90,5.30,258.00,"1,015.20",28.20,9.90,0,0.00,0,10,Partially cloudy,partly-cloudy-night,"KORD,KMDW,F1983,KPWK"
4,ORD,2024-04-18 04:00:00,49.30,46.60,42.20,76.33,0.00,0.00,NaN,0,0,10.30,6.40,282.00,"1,015.90",22.10,9.90,0,0.00,0,10,Partially cloudy,partly-cloudy-night,"KORD,KMDW,F1983,KPWK"


{'ORD', 'MCO', 'SYR', 'JFK'}


In [14]:
weather_columns_to_drop = ['dew','humidity','snowdepth','sealevelpressure', 'solarradiation', 'solarenergy','uvindex','icon','stations']
weather_data.drop(columns = weather_columns_to_drop, inplace=True)

In [15]:
# Create two data frame for origin and destination. Also rename the columns by adding the prefix.
rename_origin = {}
rename_dest = {}
for col in weather_data.columns:
    rename_origin[col] = 'ORGIN_WTH_' + col
    rename_dest[col] = 'DEST_WTH_' + col
    
org_weather_data = weather_data.rename(columns=rename_origin)
dst_weather_data = weather_data.rename(columns=rename_dest)

In [16]:
# Adding Join columns
initial_predictions_dataset['ORGIN_WTH_JOIN'] = initial_predictions_dataset['SCH_DEP_TIME'].dt.round('H').astype(str) + initial_predictions_dataset['ORIGIN']
initial_predictions_dataset['DEST_WTH_JOIN'] = initial_predictions_dataset['SCH_ARR_TIME'].dt.round('H').astype(str) + initial_predictions_dataset['DEST']
org_weather_data['ORGIN_WTH_JOIN'] = org_weather_data['ORGIN_WTH_datetime'].astype(str) + org_weather_data['ORGIN_WTH_name']
dst_weather_data['DEST_WTH_JOIN'] = dst_weather_data['DEST_WTH_datetime'].astype(str) + dst_weather_data['DEST_WTH_name']
initial_predictions_dataset.head()
org_weather_data.head()
dst_weather_data.head()

,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME,DAY_OF_WEEK,DEST,ORGIN_WTH_JOIN,DEST_WTH_JOIN
0,UA 1400,UA,UA,ORD,NaN,NaN,NaN,NaN,2024-04-19 18:52:00,2024-04-19 21:47:00,5,SYR,2024-04-19 19:00:00ORD,2024-04-19 22:00:00SYR
1,AA 3402,AA,MQ,ORD,NaN,NaN,NaN,NaN,2024-04-19 19:59:00,2024-04-19 22:52:00,5,SYR,2024-04-19 20:00:00ORD,2024-04-19 23:00:00SYR
2,B6 116,B6,B6,JFK,NaN,NaN,NaN,NaN,2024-04-19 13:34:00,2024-04-19 14:51:00,5,SYR,2024-04-19 14:00:00JFK,2024-04-19 15:00:00SYR
3,DL 5182,DL,9E,JFK,NaN,NaN,NaN,NaN,2024-04-19 14:55:00,2024-04-19 16:21:00,5,SYR,2024-04-19 15:00:00JFK,2024-04-19 16:00:00SYR
4,WN 5285,WN,WN,MCO,NaN,NaN,NaN,NaN,2024-04-19 11:35:00,2024-04-19 14:20:00,5,SYR,2024-04-19 12:00:00MCO,2024-04-19 14:00:00SYR


,ORGIN_WTH_name,ORGIN_WTH_datetime,ORGIN_WTH_temp,ORGIN_WTH_feelslike,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_preciptype,ORGIN_WTH_snow,ORGIN_WTH_windgust,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_severerisk,ORGIN_WTH_conditions,ORGIN_WTH_JOIN
0,ORD,2024-04-18 00:00:00,55.70,55.70,0.00,0.00,NaN,0,25.30,12.40,301.00,68.00,9.90,10,Partially cloudy,2024-04-18 00:00:00ORD
1,ORD,2024-04-18 01:00:00,53.80,53.80,0.00,0.00,NaN,0,20.80,8.20,290.00,32.40,9.90,10,Partially cloudy,2024-04-18 01:00:00ORD
2,ORD,2024-04-18 02:00:00,51.30,51.30,0.00,0.00,NaN,0,17.20,3.70,267.00,22.10,9.90,10,Partially cloudy,2024-04-18 02:00:00ORD
3,ORD,2024-04-18 03:00:00,49.80,47.80,0.00,0.00,NaN,0,13.90,5.30,258.00,28.20,9.90,10,Partially cloudy,2024-04-18 03:00:00ORD
4,ORD,2024-04-18 04:00:00,49.30,46.60,0.00,0.00,NaN,0,10.30,6.40,282.00,22.10,9.90,10,Partially cloudy,2024-04-18 04:00:00ORD


,DEST_WTH_name,DEST_WTH_datetime,DEST_WTH_temp,DEST_WTH_feelslike,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_preciptype,DEST_WTH_snow,DEST_WTH_windgust,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_severerisk,DEST_WTH_conditions,DEST_WTH_JOIN
0,ORD,2024-04-18 00:00:00,55.70,55.70,0.00,0.00,NaN,0,25.30,12.40,301.00,68.00,9.90,10,Partially cloudy,2024-04-18 00:00:00ORD
1,ORD,2024-04-18 01:00:00,53.80,53.80,0.00,0.00,NaN,0,20.80,8.20,290.00,32.40,9.90,10,Partially cloudy,2024-04-18 01:00:00ORD
2,ORD,2024-04-18 02:00:00,51.30,51.30,0.00,0.00,NaN,0,17.20,3.70,267.00,22.10,9.90,10,Partially cloudy,2024-04-18 02:00:00ORD
3,ORD,2024-04-18 03:00:00,49.80,47.80,0.00,0.00,NaN,0,13.90,5.30,258.00,28.20,9.90,10,Partially cloudy,2024-04-18 03:00:00ORD
4,ORD,2024-04-18 04:00:00,49.30,46.60,0.00,0.00,NaN,0,10.30,6.40,282.00,22.10,9.90,10,Partially cloudy,2024-04-18 04:00:00ORD


In [17]:
# Join data set
merged_flight_weather = pd.merge(initial_predictions_dataset, org_weather_data, on='ORGIN_WTH_JOIN')
merged_flight_weather = pd.merge(merged_flight_weather, dst_weather_data, on='DEST_WTH_JOIN')

merged_flight_weather.head()
merged_flight_weather.shape
merged_flight_weather.columns

,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME,DAY_OF_WEEK,DEST,ORGIN_WTH_JOIN,DEST_WTH_JOIN,ORGIN_WTH_name,ORGIN_WTH_datetime,ORGIN_WTH_temp,ORGIN_WTH_feelslike,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_preciptype,ORGIN_WTH_snow,ORGIN_WTH_windgust,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_severerisk,ORGIN_WTH_conditions,DEST_WTH_name,DEST_WTH_datetime,DEST_WTH_temp,DEST_WTH_feelslike,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_preciptype,DEST_WTH_snow,DEST_WTH_windgust,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_severerisk,DEST_WTH_conditions
0,UA 1400,UA,UA,ORD,NaN,NaN,NaN,NaN,2024-04-19 18:52:00,2024-04-19 21:47:00,5,SYR,2024-04-19 19:00:00ORD,2024-04-19 22:00:00SYR,ORD,2024-04-19 19:00:00,52.10,52.10,0.00,0.00,NaN,0,20.80,12.80,270.00,46.00,10.10,10,Partially cloudy,SYR,2024-04-19 22:00:00,53.00,53.00,0.00,34.00,NaN,0,18.30,12.80,270.00,57.00,10.10,10,Partially cloudy
1,AA 3402,AA,MQ,ORD,NaN,NaN,NaN,NaN,2024-04-19 19:59:00,2024-04-19 22:52:00,5,SYR,2024-04-19 20:00:00ORD,2024-04-19 23:00:00SYR,ORD,2024-04-19 20:00:00,49.90,45.10,0.00,0.00,NaN,0,19.50,12.80,270.00,48.00,10.10,10,Partially cloudy,SYR,2024-04-19 23:00:00,52.10,52.10,0.00,26.00,NaN,0,18.30,11.40,270.00,48.00,10.10,10,Partially cloudy
2,B6 116,B6,B6,JFK,NaN,NaN,NaN,NaN,2024-04-19 13:34:00,2024-04-19 14:51:00,5,SYR,2024-04-19 14:00:00JFK,2024-04-19 15:00:00SYR,JFK,2024-04-19 14:00:00,54.10,54.10,0.00,9.00,NaN,0,18.30,12.80,150.00,37.00,10.10,10,Partially cloudy,SYR,2024-04-19 15:00:00,60.00,60.00,0.01,74.00,rain,0,33.30,21.90,190.00,85.00,10.10,10,"Rain, Partially cloudy"
3,DL 5182,DL,9E,JFK,NaN,NaN,NaN,NaN,2024-04-19 14:55:00,2024-04-19 16:21:00,5,SYR,2024-04-19 15:00:00JFK,2024-04-19 16:00:00SYR,JFK,2024-04-19 15:00:00,53.00,53.00,0.00,9.00,NaN,0,17.20,11.40,150.00,51.00,10.10,10,Partially cloudy,SYR,2024-04-19 16:00:00,60.00,60.00,0.01,75.00,rain,0,34.40,21.90,190.00,80.00,6.90,10,"Rain, Partially cloudy"
4,B6 656,B6,B6,MCO,NaN,NaN,NaN,NaN,2024-04-19 13:35:00,2024-04-19 16:25:00,5,SYR,2024-04-19 14:00:00MCO,2024-04-19 16:00:00SYR,MCO,2024-04-19 14:00:00,89.00,89.90,0.00,3.00,NaN,0,6.90,5.80,270.00,20.00,10.10,10,Clear,SYR,2024-04-19 16:00:00,60.00,60.00,0.01,75.00,rain,0,34.40,21.90,190.00,80.00,6.90,10,"Rain, Partially cloudy"


(23, 44)

Index(['FLIGHT NUMBER', 'MKT_UNIQUE_CARRIER', 'OP_UNIQUE_CARRIER', 'ORIGIN',
       'ARRIVAL STATUS', 'ARRIVAL STATUS_Prev_flight_early',
       'ARRIVAL STATUS_Prev_flight_ontime', 'ARRIVAL STATUS_Prev_flight_late',
       'SCH_DEP_TIME', 'SCH_ARR_TIME', 'DAY_OF_WEEK', 'DEST', 'ORGIN_WTH_JOIN',
       'DEST_WTH_JOIN', 'ORGIN_WTH_name', 'ORGIN_WTH_datetime',
       'ORGIN_WTH_temp', 'ORGIN_WTH_feelslike', 'ORGIN_WTH_precip',
       'ORGIN_WTH_precipprob', 'ORGIN_WTH_preciptype', 'ORGIN_WTH_snow',
       'ORGIN_WTH_windgust', 'ORGIN_WTH_windspeed', 'ORGIN_WTH_winddir',
       'ORGIN_WTH_cloudcover', 'ORGIN_WTH_visibility', 'ORGIN_WTH_severerisk',
       'ORGIN_WTH_conditions', 'DEST_WTH_name', 'DEST_WTH_datetime',
       'DEST_WTH_temp', 'DEST_WTH_feelslike', 'DEST_WTH_precip',
       'DEST_WTH_precipprob', 'DEST_WTH_preciptype', 'DEST_WTH_snow',
       'DEST_WTH_windgust', 'DEST_WTH_windspeed', 'DEST_WTH_winddir',
       'DEST_WTH_cloudcover', 'DEST_WTH_visibility', 'DEST_WTH_severeris

In [18]:
# Drop unnecessary columns
merged_flight_weather_updated = merged_flight_weather.drop(columns=['ORGIN_WTH_JOIN', 'DEST_WTH_JOIN', 'ORGIN_WTH_datetime', 'DEST_WTH_datetime', 'ORGIN_WTH_name', 'DEST_WTH_name'])

In [19]:
merged_flight_weather_updated.head()

,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME,DAY_OF_WEEK,DEST,ORGIN_WTH_temp,ORGIN_WTH_feelslike,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_preciptype,ORGIN_WTH_snow,ORGIN_WTH_windgust,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_severerisk,ORGIN_WTH_conditions,DEST_WTH_temp,DEST_WTH_feelslike,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_preciptype,DEST_WTH_snow,DEST_WTH_windgust,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_severerisk,DEST_WTH_conditions
0,UA 1400,UA,UA,ORD,NaN,NaN,NaN,NaN,2024-04-19 18:52:00,2024-04-19 21:47:00,5,SYR,52.10,52.10,0.00,0.00,NaN,0,20.80,12.80,270.00,46.00,10.10,10,Partially cloudy,53.00,53.00,0.00,34.00,NaN,0,18.30,12.80,270.00,57.00,10.10,10,Partially cloudy
1,AA 3402,AA,MQ,ORD,NaN,NaN,NaN,NaN,2024-04-19 19:59:00,2024-04-19 22:52:00,5,SYR,49.90,45.10,0.00,0.00,NaN,0,19.50,12.80,270.00,48.00,10.10,10,Partially cloudy,52.10,52.10,0.00,26.00,NaN,0,18.30,11.40,270.00,48.00,10.10,10,Partially cloudy
2,B6 116,B6,B6,JFK,NaN,NaN,NaN,NaN,2024-04-19 13:34:00,2024-04-19 14:51:00,5,SYR,54.10,54.10,0.00,9.00,NaN,0,18.30,12.80,150.00,37.00,10.10,10,Partially cloudy,60.00,60.00,0.01,74.00,rain,0,33.30,21.90,190.00,85.00,10.10,10,"Rain, Partially cloudy"
3,DL 5182,DL,9E,JFK,NaN,NaN,NaN,NaN,2024-04-19 14:55:00,2024-04-19 16:21:00,5,SYR,53.00,53.00,0.00,9.00,NaN,0,17.20,11.40,150.00,51.00,10.10,10,Partially cloudy,60.00,60.00,0.01,75.00,rain,0,34.40,21.90,190.00,80.00,6.90,10,"Rain, Partially cloudy"
4,B6 656,B6,B6,MCO,NaN,NaN,NaN,NaN,2024-04-19 13:35:00,2024-04-19 16:25:00,5,SYR,89.00,89.90,0.00,3.00,NaN,0,6.90,5.80,270.00,20.00,10.10,10,Clear,60.00,60.00,0.01,75.00,rain,0,34.40,21.90,190.00,80.00,6.90,10,"Rain, Partially cloudy"


In [20]:
merged_flight_weather_updated.columns

Index(['FLIGHT NUMBER', 'MKT_UNIQUE_CARRIER', 'OP_UNIQUE_CARRIER', 'ORIGIN',
       'ARRIVAL STATUS', 'ARRIVAL STATUS_Prev_flight_early',
       'ARRIVAL STATUS_Prev_flight_ontime', 'ARRIVAL STATUS_Prev_flight_late',
       'SCH_DEP_TIME', 'SCH_ARR_TIME', 'DAY_OF_WEEK', 'DEST', 'ORGIN_WTH_temp',
       'ORGIN_WTH_feelslike', 'ORGIN_WTH_precip', 'ORGIN_WTH_precipprob',
       'ORGIN_WTH_preciptype', 'ORGIN_WTH_snow', 'ORGIN_WTH_windgust',
       'ORGIN_WTH_windspeed', 'ORGIN_WTH_winddir', 'ORGIN_WTH_cloudcover',
       'ORGIN_WTH_visibility', 'ORGIN_WTH_severerisk', 'ORGIN_WTH_conditions',
       'DEST_WTH_temp', 'DEST_WTH_feelslike', 'DEST_WTH_precip',
       'DEST_WTH_precipprob', 'DEST_WTH_preciptype', 'DEST_WTH_snow',
       'DEST_WTH_windgust', 'DEST_WTH_windspeed', 'DEST_WTH_winddir',
       'DEST_WTH_cloudcover', 'DEST_WTH_visibility', 'DEST_WTH_severerisk',
       'DEST_WTH_conditions'],
      dtype='object')

In [21]:
columns_to_drop = ['FLIGHT NUMBER', 'ARRIVAL STATUS', 'ARRIVAL STATUS_Prev_flight_early', 'ARRIVAL STATUS_Prev_flight_ontime', 'ARRIVAL STATUS_Prev_flight_late', 'ORGIN_WTH_feelslike',
                    'ORGIN_WTH_windgust', 'DEST_WTH_feelslike', 'DEST_WTH_windgust',
                    'DEST', 'DEST_WTH_preciptype', 'ORGIN_WTH_preciptype',
                    'DEST_WTH_conditions', 'ORGIN_WTH_conditions']
flight_weather_data_updated =  merged_flight_weather_updated.drop(columns = columns_to_drop)
flight_weather_data_updated.shape
flight_weather_data_updated.columns

(23, 24)

Index(['MKT_UNIQUE_CARRIER', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'SCH_DEP_TIME',
       'SCH_ARR_TIME', 'DAY_OF_WEEK', 'ORGIN_WTH_temp', 'ORGIN_WTH_precip',
       'ORGIN_WTH_precipprob', 'ORGIN_WTH_snow', 'ORGIN_WTH_windspeed',
       'ORGIN_WTH_winddir', 'ORGIN_WTH_cloudcover', 'ORGIN_WTH_visibility',
       'ORGIN_WTH_severerisk', 'DEST_WTH_temp', 'DEST_WTH_precip',
       'DEST_WTH_precipprob', 'DEST_WTH_snow', 'DEST_WTH_windspeed',
       'DEST_WTH_winddir', 'DEST_WTH_cloudcover', 'DEST_WTH_visibility',
       'DEST_WTH_severerisk'],
      dtype='object')

In [22]:
flight_weather_data_updated.to_csv('./test_data/Test_Merged_Data.csv', index=False)